---
title: "Listing Generation Code"
---

This is based on [Fernando's geojupyter blog generator](https://github.com/fperez/geojupyter.org/blob/myst/blog/mkindex-blog.ipynb).

Executing this notebook created a listing output file which contains the contents of 

In [40]:
# Title of listing page
listing_title = "MOTD"

# Header content included in listing page
listing_header = "header.md"

# File name of listing page
listing_filename = "index.md"

# Path to markdown documents to be included in listing
listing_path = "."

In [35]:
import os
import yaml
import re
from pathlib import Path
from datetime import datetime

def extract_frontmatter(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
    match = re.match(r'^---\s*\n(.*?)\n---\s*\n', content, re.DOTALL)
    if match:
        frontmatter = yaml.safe_load(match.group(1))
        return frontmatter
    return {}

def format_author(author_field):
    if isinstance(author_field, dict):
        return author_field.get("name", "").strip()
    elif isinstance(author_field, list):
        names = [a.get("name", "") if isinstance(a, dict) else str(a) for a in author_field]
        return ", ".join(n for n in names if n).strip()
    elif isinstance(author_field, str):
        return author_field.strip()
    return ""

def parse_date(date_str):
    try:
        return datetime.fromisoformat(date_str)
    except Exception:
        return None

def generate_cards(base_dir, order="reverse"):
    card_data = []

    for root, dirs, files in os.walk(base_dir):
        # Skip only top-level index.md
        if Path(root).resolve() == Path(base_dir).resolve() and 'index.md' in files:
            files.remove('index.md')

        if 'index.md' in files:
            index_path = os.path.join(root, 'index.md')
            fm = extract_frontmatter(index_path)
            if not fm:
                continue

            rel_folder = os.path.relpath(root, base_dir)

            title = fm.get('title', 'Untitled')
            date_str = fm.get('date', '1900-01-01')
            date_obj = parse_date(date_str)
            description = fm.get('description', '').strip()
            author = format_author(fm.get('author', ''))

            link_path = os.path.relpath(index_path, base_dir)

            card_lines = [
                f"# {date_str} {title}", "",
                f"::::{{card}}",
                f":url: {link_path}",
            ]
            if author:
                card_lines.append(f":footer: {author}")
            card_lines.append("")

            if description:
                card_lines.append("")
                card_lines.append(description)
            card_lines.append("")

            # include listing document
            card_lines.append(f"```{{include}} {index_path}\n```")            
            card_lines.append("::::\n")
            
            card_text = "\n".join(card_lines)
            card_data.append((date_obj, card_text))

    # Sort by parsed date
    card_data.sort(
        key=lambda x: x[0] if x[0] else datetime.min,
        reverse=(order == "reverse")
    )

    return [text for _, text in card_data]


In [36]:
# Let's use it and do a quick visual sanity check
cards = generate_cards(listing_path)
print(cards[1])

# 2025-04-25 Brief network maintenance

::::{card}
:url: 2025-04-25-network/index.md
:footer: Ryan Lovett


```{include} ./2025-04-25-network/index.md
```
::::



In [41]:
# Auto-generate the listing index page.
header = f"""\
---
title: "{listing_title}"
---

```{{include}} {listing_header}
```
"""

nc = len(cards)
with open(listing_filename, "w") as f:
    f.write(header)
    for n, c in enumerate(cards):
        f.write("\n% ===================================\n")
        f.write(f"% auto-generated card #{n+1}/{nc}\n\n")
        f.writelines(c)
        